In [173]:
import pandas as pd
import numpy as np
import re

In [174]:
# Read in products

df = pd.read_csv('messy_products.csv')

df

,Product_ID,Product_Name,Category,Price,Quantity,Rating,Release_Date,Weight_kg,Screen_Size_cm,Manufacturer,In_Stock
0,101,Galaxy S21,Smartphone,799.0,10.0,4.5,1/29/2021,0.171,15.2,Samsung,Yes
1,102,iPhone 12,Smartphone,NaN,5.0,4.7,12/10/2020,0.164,15.4,Apple,yes
2,103,Pixel 5,Smartphone,699.0,NaN,4.6,10/15/2020,0.151,15,Google,TRUE
3,104,ThinkPad X1,Laptop,1399.0,3.0,4.8,3/1/2021,1.130,35.6,Lenovo,Y
4,105,NaN,Laptop,999.0,7.0,4.3,8/20/2020,1.250,39.6,HP,No
5,106,MacBook Pro,Laptop,2399.0,2.0,NaN,5/21/2021,1.400,34,Apple,N
6,107,Surface Pro,Tablet,899.0,0.0,4.4,2/2/2021,0.770,31.2,Microsoft,FALSE
7,108,iPad Air,Tablet,599.0,12.0,4.6,March 18 2021,0.458,27.9,Apple,TRUE
8,109,Fire HD 10,tablet,149.0,25.0,3.9,9/30/2020,0.465,25.7,Amazon,Yes
9,110,Galaxy Tab S7,Tablet,649.0,8.0,4.5,1/1/2021,0.498,27.8,Samsung,1


In [175]:
df.isna().sum()

Product_ID        0
Product_Name      1
Category          0
Price             2
Quantity          1
Rating            1
Release_Date      0
Weight_kg         1
Screen_Size_cm    0
Manufacturer      0
In_Stock          0
dtype: int64

In [176]:
df.duplicated().sum()

np.int64(0)

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Product_ID      12 non-null     int64  
 1   Product_Name    11 non-null     object 
 2   Category        12 non-null     object 
 3   Price           10 non-null     float64
 4   Quantity        11 non-null     float64
 5   Rating          11 non-null     float64
 6   Release_Date    12 non-null     object 
 7   Weight_kg       11 non-null     float64
 8   Screen_Size_cm  12 non-null     object 
 9   Manufacturer    12 non-null     object 
 10  In_Stock        12 non-null     object 
dtypes: float64(4), int64(1), object(6)
memory usage: 1.2+ KB


### Data Cleaning steps/observations
<p>Product ID = 112 rating out of scale and negative quantity should remove </p>
<p>Missing values: Price(Average price per categroy), Quantity (set to 0), Rating (fill with Median), Product_name (Combine Manufacturer and Category) </p>
<p>Inconsistent columns: Category(Tablet vs tablet), In_Stock (inconsistent), Release_Date (Invalid date format) </p>
<p>Adding units: Weight_kg (to lbs), Screen_Size_cm (to inch) </p>
<p>Dtype: Quantity (float -> int), Screen_Size_cm (object -> float) </p>

In [178]:
# Quantity missing to 0's and convert to int
df.loc[:, 'Quantity'] = df['Quantity'].fillna(0)
# Remove quantity < 0
df = df[df['Quantity'] >= 0]

# Quantity change to float to intt
df['Quantity'] = df['Quantity'].astype(int)

# Price average of category column
df.loc[:, 'Price'] = df.groupby('Category')['Price'].transform(lambda x: x.fillna(x.mean()))

# Rating median value
df.loc[:, 'Rating'] = df['Rating'].transform(lambda x: x.fillna(x.median()))

# Screen Size change to float type from object
df['Screen_Size_cm'] = df['Screen_Size_cm'].astype(float)

# Add Inch/lbs col for Screen size and weighht
df['Screen_Size_inch'] = (df['Screen_Size_cm'] * 0.393701).round(3)
df['Weight_lbs'] = (df['Weight_kg']*2.2046).round(3)

# Release Date format dates 
df.loc[:, 'Release_Date'] = pd.to_datetime(df['Release_Date'], format='mixed').dt.date

# title casing Category column for consistency
df.loc[:, 'Category'] = df['Category'].str.title()

# Missing from product name Manufacturer + Category
df.loc[:, 'Product_Name'] = df['Product_Name'].fillna(df['Manufacturer'] + " " + df['Category'])

df


C:\Users\brooksat\AppData\Local\Temp\ipykernel_828\1745468463.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Quantity'] = df['Quantity'].astype(int)
C:\Users\brooksat\AppData\Local\Temp\ipykernel_828\1745468463.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Screen_Size_cm'] = df['Screen_Size_cm'].astype(float)
C:\Users\brooksat\AppData\Local\Temp\ipykernel_828\1745468463.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,Product_ID,Product_Name,Category,Price,Quantity,Rating,Release_Date,Weight_kg,Screen_Size_cm,Manufacturer,In_Stock,Screen_Size_inch,Weight_lbs
0,101,Galaxy S21,Smartphone,799.0,10,4.5,2021-01-29,0.171,15.2,Samsung,Yes,5.984,0.377
1,102,iPhone 12,Smartphone,749.0,5,4.7,2020-12-10,0.164,15.4,Apple,yes,6.063,0.362
2,103,Pixel 5,Smartphone,699.0,0,4.6,2020-10-15,0.151,15.0,Google,TRUE,5.906,0.333
3,104,ThinkPad X1,Laptop,1399.0,3,4.8,2021-03-01,1.130,35.6,Lenovo,Y,14.016,2.491
4,105,HP Laptop,Laptop,999.0,7,4.3,2020-08-20,1.250,39.6,HP,No,15.591,2.756
5,106,MacBook Pro,Laptop,2399.0,2,4.5,2021-05-21,1.400,34.0,Apple,N,13.386,3.086
6,107,Surface Pro,Tablet,899.0,0,4.4,2021-02-02,0.770,31.2,Microsoft,FALSE,12.283,1.698
7,108,iPad Air,Tablet,599.0,12,4.6,2021-03-18,0.458,27.9,Apple,TRUE,10.984,1.010
8,109,Fire HD 10,Tablet,149.0,25,3.9,2020-09-30,0.465,25.7,Amazon,Yes,10.118,1.025
9,110,Galaxy Tab S7,Tablet,649.0,8,4.5,2021-01-01,0.498,27.8,Samsung,1,10.945,1.098


In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 0 to 10
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_ID        11 non-null     int64  
 1   Product_Name      11 non-null     object 
 2   Category          11 non-null     object 
 3   Price             11 non-null     float64
 4   Quantity          11 non-null     int64  
 5   Rating            11 non-null     float64
 6   Release_Date      11 non-null     object 
 7   Weight_kg         11 non-null     float64
 8   Screen_Size_cm    11 non-null     float64
 9   Manufacturer      11 non-null     object 
 10  In_Stock          11 non-null     object 
 11  Screen_Size_inch  11 non-null     float64
 12  Weight_lbs        11 non-null     float64
dtypes: float64(6), int64(2), object(5)
memory usage: 1.2+ KB


In [180]:
df.loc[:, 'In_Stock'] = np.where(df['In_Stock'].str.upper().str.contains(r'(Y|TRUE|1)', regex=True), "Y", "N")

df

C:\Users\brooksat\AppData\Local\Temp\ipykernel_828\1663810098.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[:, 'In_Stock'] = np.where(df['In_Stock'].str.upper().str.contains(r'(Y|TRUE|1)', regex=True), "Y", "N")


,Product_ID,Product_Name,Category,Price,Quantity,Rating,Release_Date,Weight_kg,Screen_Size_cm,Manufacturer,In_Stock,Screen_Size_inch,Weight_lbs
0,101,Galaxy S21,Smartphone,799.0,10,4.5,2021-01-29,0.171,15.2,Samsung,Y,5.984,0.377
1,102,iPhone 12,Smartphone,749.0,5,4.7,2020-12-10,0.164,15.4,Apple,Y,6.063,0.362
2,103,Pixel 5,Smartphone,699.0,0,4.6,2020-10-15,0.151,15.0,Google,Y,5.906,0.333
3,104,ThinkPad X1,Laptop,1399.0,3,4.8,2021-03-01,1.130,35.6,Lenovo,Y,14.016,2.491
4,105,HP Laptop,Laptop,999.0,7,4.3,2020-08-20,1.250,39.6,HP,N,15.591,2.756
5,106,MacBook Pro,Laptop,2399.0,2,4.5,2021-05-21,1.400,34.0,Apple,N,13.386,3.086
6,107,Surface Pro,Tablet,899.0,0,4.4,2021-02-02,0.770,31.2,Microsoft,N,12.283,1.698
7,108,iPad Air,Tablet,599.0,12,4.6,2021-03-18,0.458,27.9,Apple,Y,10.984,1.010
8,109,Fire HD 10,Tablet,149.0,25,3.9,2020-09-30,0.465,25.7,Amazon,Y,10.118,1.025
9,110,Galaxy Tab S7,Tablet,649.0,8,4.5,2021-01-01,0.498,27.8,Samsung,Y,10.945,1.098


In [182]:
# Read to csv
df.to_csv('clean_products.csv', index=False, header=True)